# Body movement detection and analysis

#### The application uses MediaPipe and OpenCV to capture video from a webcam feed to detect and analyse movement. The primary objective is to assess body movement in real time, categorise activity levels and identify the most active body parts.


## Libraries and requirements

#### Import libraries

In [1]:
# Import libraries
import numpy as np
import cv2
import mediapipe as mp
import time

#### Application requirements

In [2]:
# Libraries and versions
libraries = {
    'numpy': np.__version__,
    'opencv-python': cv2.__version__,
    'mediapipe': mp.__version__,
}

# Write to the requirements file and print
with open('requirements.txt', 'w') as f:
    for lib, version in libraries.items():
        f.write(f"{lib}=={version}\n")
        print(f"{lib}: {version}") 

numpy: 1.26.4
opencv-python: 4.10.0
mediapipe: 0.10.14


### Body movement detection and analysis

#### Functions for calculating and categorising movement for body parts

In [3]:
# Function to analyze movements based on landmarks
def calculate_movement(landmarks_curr, landsmarks_prev):
    # Initialize dictionary for distances 
    movement = {}
    
    # Body parts tracked
    body_parts = {
        'Left Eye': (mp_pose.PoseLandmark.LEFT_EYE.value),  # Left eye landmark (used for tracking head movement)
        'Right Eye': (mp_pose.PoseLandmark.RIGHT_EYE.value),  # Right eye landmark (used for tracking head movement)
        #'Nose ': (mp_pose.PoseLandmark.NOSE.value),  #Nose landmark (not in use)
        'Left Shoulder': mp_pose.PoseLandmark.LEFT_SHOULDER.value,  # Left Shoulder
        'Right Shoulder': mp_pose.PoseLandmark.RIGHT_SHOULDER.value,  # Right Shoulder
        'Left Elbow': mp_pose.PoseLandmark.LEFT_ELBOW.value,  # Left Elbow
        'Right Elbow': mp_pose.PoseLandmark.RIGHT_ELBOW.value,  # Right Elbow
        'Left Wrist': mp_pose.PoseLandmark.LEFT_WRIST.value, # Left wrist
        'Right Wrist': mp_pose.PoseLandmark.RIGHT_WRIST.value, # Right wrist
        'Left Hip': mp_pose.PoseLandmark.LEFT_HIP.value,  # Left Hip
        'Right Hip': mp_pose.PoseLandmark.RIGHT_HIP.value,  # Right Hip
        'Left Knee': mp_pose.PoseLandmark.LEFT_KNEE.value,  # Left Knee
        'Right Knee': mp_pose.PoseLandmark.RIGHT_KNEE.value,  # Right Knee
        'Left Ankle': mp_pose.PoseLandmark.LEFT_ANKLE.value,  # Left Ankle
        'Right Ankle': mp_pose.PoseLandmark.RIGHT_ANKLE.value  # Right Ankle
    }
    
    # Calculate movement for each tracked body part
    for part, i in body_parts.items():
        pos_curr = [landmarks_curr[i].x, landmarks_curr[i].y]
        pos_prev = [landsmarks_prev[i].x, landsmarks_prev[i].y]
        distance = np.linalg.norm(np.array(pos_curr) - np.array(pos_prev))
        movement[part] = distance

    # Return distances dictionary
    return movement

In [4]:
# Function to categorize overall body movement
def categorize_movement(total_movement):
    if total_movement < 0.1:  # Low body movement observed during video capture (very little movement observed)
        return "Low", total_movement
    elif total_movement < 0.4:  # Medium body movement observed during video capture (regular movement observed)
        return "Medium", total_movement
    else:  # High body movement observed during video capture (excessive movement observed)
        return "High", total_movement

#### Analysing webcam capture

In [5]:
# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Initialize previous landmarks and analytics storage
landsmarks_prev = None

# Initialize dictionary for movement data
movement_data = {}

# Initialize time variables for identifying time of observations
start_time = time.time()  # Start time
max_time = 0  # Variable for time of maximum movement
max_movement = 0  # Variable for maximum movement score observed

# Initialize video capture from webcam
video = cv2.VideoCapture(0)

while video.isOpened():
    ret, image = video.read()
    if not ret:
        break

    # Convert image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Process image to retrieve pose landmarks
    image_pose = pose.process(image_rgb)
    
    # Check if landmarks are detected
    if image_pose.pose_landmarks:
        # Draw detected landmarks 
        mp_drawing.draw_landmarks(
            image, image_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Extract current landmark
        landmarks_curr = image_pose.pose_landmarks.landmark

        # Detect and analyse body part movement obtained with the landmarks
        if landsmarks_prev is not None:
            # Call movement calculator function to get differrence between current and prevous landmark
            movement = calculate_movement(landmarks_curr, landsmarks_prev)
            movement_total = sum(movement.values())  # Total movement score

            # Call movement categorizer function to obtain total movement classification score (low, med, high) 
            movement_class, _ = categorize_movement(movement_total)

            # Summarize body part movement
            movement_parts = {
                'Head': movement['Left Eye'] + movement['Left Eye'],  # Head movement score
                'Shoulders': movement['Left Shoulder'] + movement['Right Shoulder'],  # Shoulders movement score
                'Hands': movement['Left Wrist'] + movement['Right Wrist'],  # Hands movement score
                'Elbows': movement['Left Elbow'] + movement['Right Elbow'],  # Elbows movement score
                'Hips': movement['Left Hip'] + movement['Right Hip'],  # Hips movement score
                'Knees': movement['Left Knee'] + movement['Right Knee'],  # Knees movement score
                'Ankles': movement['Left Ankle'] + movement['Right Ankle']  # Ankles movement score
            }
            
            # Identify most moved body part
            movement_max = max(movement_parts, key=movement_parts.get)

            # Check if this is the maximum movement so far
            current_time = time.time() - start_time
        
            # Check if current overall movement score is higher than the previous max observer
            if movement_total > max_movement:
                max_movement = movement_total  # Update if new max is observed
                max_time = current_time  # Update obervation time

            # Display overall movement classification score on video capture
            cv2.putText(
                image, f'Movement: {movement_class}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA
            )
            
            # Display time of maximum movement observed
            cv2.putText(
                image, f'Max Movement at: {max_time:.2f} sec', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA
            )

            # Store the movement data to the dictionary 
            movement_data['total_movement'] = movement_total  # Overall movement score (i.e. sum of movement values)
            movement_data['total_movement_class'] = movement_class  # Overall movement classification
            movement_data['most_moved_part'] = movement_max  # Most moved body part class
            movement_data['most_moved_score'] = movement_parts[movement_max]  # Most moved body part score
            movement_data['individual_movements'] = movement  # Individual body part scores




        # Update previous landmarks
        landsmarks_prev = landmarks_curr

    # Display image with landmarks
    cv2.imshow('Pose Estimation & Movement Analysis', image)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
video.release()
cv2.destroyAllWindows()

# Print the analytical report after closing the video capture
print("Movement Analysis Report:") 
print(f"Overall Movement Score: {movement_data.get('total_movement', 0):.2f}")  # Overall score
print(f"Overall Movement Classification: {movement_data.get('total_movement_class', 'N/A')}") # Overall classification
print(f'Maximum Movement Observed at: {max_time:.2f} sec')
print(f"Most Moved Body Part: {movement_data.get('most_moved_part', 'N/A')}")  # Most moved body part class
print(f"Most Moved Body Part Score: {movement_data.get('most_moved_score', 0):.2f}")  # Most moved body part score
# All individual body part scores observed
print("Individual Movements:") 
for part, movement in movement_data.get('individual_movements', {}).items():
    print(f"  {part}: {movement:.2f}")


Movement Analysis Report:
Overall Movement Score: 0.32
Overall Movement Classification: Medium
Maximum Movement Observed at: 23.20 sec
Most Moved Body Part: Elbows
Most Moved Body Part Score: 0.13
Individual Movements:
  Left Eye: 0.00
  Right Eye: 0.00
  Left Shoulder: 0.01
  Right Shoulder: 0.01
  Left Elbow: 0.12
  Right Elbow: 0.02
  Left Wrist: 0.03
  Right Wrist: 0.03
  Left Hip: 0.01
  Right Hip: 0.01
  Left Knee: 0.02
  Right Knee: 0.02
  Left Ankle: 0.03
  Right Ankle: 0.03


## Conclusion

The application effectively tracks body part movements and generates analytics, including which body part is the most active, the overall movement classification, and the specific timepoints of heightened activity. While the insights from movement analysis alone may not yield significant meaning, when combined with other emotional recognition tools—such as Voice Tone Analysis (emotion detection from audio), Semantic Analysis of Spoken Words (using NLP), and Facial Expression Analysis (emotion detection through facial features)—the application can provide valuable insights into body language during critical moments. This multifaceted approach enhances the understanding of emotional and behavioral patterns, making it a useful too